# Napoleon’s March
> This notebook attempts to recreate one of the most celebrated examples of data visualization in history, Charles Joseph Minard's [iconic chart](https://upload.wikimedia.org/wikipedia/commons/2/29/Minard.png) of Napoleon's disastrous Russian campaign of 1812, created in 1869. I sought to make it with Python and the Altair charting library. Points and figures on the map are approximated thanks to data from [@andrewheiss](https://github.com/andrewheiss/fancy-minard).

#### Import Python tools and Jupyter configuration

In [1]:
import altair as alt
import numpy as np
import pandas as pd
import jupyter_black
import geopandas as gpd

In [2]:
jupyter_black.load()
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

---

## Read data

### Cities

In [3]:
cities = pd.read_csv(
    "https://raw.githubusercontent.com/andrewheiss/fancy-minard/master/input/minard/cities.txt",
    sep="\s+",
    engine="python",
)

In [4]:
cities.head()

,long,lat,city
0,24.0,55.0,Kowno
1,25.3,54.7,Wilna
2,26.4,54.4,Smorgoni
3,26.8,54.3,Moiodexno
4,27.7,55.2,Gloubokoe


#### Updating the cities dataframe to help with labeling

In [5]:
updates = [
    {"city": "Kowno", "long": 24.7},
    {"city": "Polotzk", "lat": 55.58},
    {"city": "Moiodexno", "lat": 54.23},
    {"city": "Smorgoni", "lat": 54.488},
    {"city": "Studienska", "lat": 54.45},
    {"city": "Bobr", "lat": 54.2, "long": 29.59},
    {"city": "Dorogobouge", "long": 32.8},
    {"city": "Wixma", "lat": 55.1, "long": 34.5},
    {"city": "Malo-Jarosewii", "lat": 54.87, "long": 36.5},
    {"city": "Moscou", "lat": 55.83},
]

for update in updates:
    city_name = update.pop("city")
    for column, new_value in update.items():
        cities.loc[cities["city"] == city_name, column] = new_value

In [6]:
new_rows = pd.DataFrame(
    {
        "city": ["START", "440k troops", "END", "10k"],
        "long": [24, 24, 24, 24],  # Replace with actual longitude values
        "lat": [55.30, 55.18, 54.3, 54.18],  # Replace with actual latitude values
    }
)

# Concatenate the original DataFrame with the new rows
cities = pd.concat([cities, new_rows], ignore_index=True)

#### Alternative spellings of city names

In [7]:
name_mapping = {
    "Kowno": "Kaunas",
    "Wilna": "Vilnius",
    "Smorgoni": "Smarhon",
    "Moiodexno": "Mstsislaw",  # Mstsislaw?
    "Gloubokoe": "Hlybokaye",
    "Minsk": "Minsk",
    "Studienska": "Studzieniczna",  # Studzieniczna?
    "Polotzk": "Polotsk",
    "Bobr": "Bobruisk",  # Bobruisk?
    "Witebsk": "Vitebsk",
    "Orscha": "Orsha",
    "Mohilow": "Mogilev",
    "Smolensk": "Smolensk",
    "Dorogobouge": "Dorogobuzh",
    "Wixma": "Vyazma",
    "Chjat": "Chyhyryn",  # Chyhyryn?
    "Mojaisk": "Mozhaysk",
    "Moscou": "Moscow",
    "Tarantino": "Tarantino",  # ?
    "Malo-Jarosewii": "Maloyaroslavets",  # ?
    "START": "START",
    "END": "END",
    "440k troops": "440k troops",
    "10k": "10k",
}

cities["city"] = cities["city"].map(name_mapping)

---

### Temperatures

In [8]:
temps = pd.read_csv(
    "https://raw.githubusercontent.com/andrewheiss/fancy-minard/master/input/minard/temps.txt",
    sep="\s+",
    engine="python",
    parse_dates=["date"],
    date_format="%d%b%Y",
).sort_values("long", ascending=False)

#### Adjust date from Reamur to Celcius

In [9]:
temps["temp_c"] = temps["temp"] * 5 / 4

---

### Troops

In [10]:
troops = pd.read_csv(
    "https://raw.githubusercontent.com/andrewheiss/fancy-minard/master/input/minard/troops.txt",
    sep="\s+",
    engine="python",
).sort_values(by=["group", "survivors"], ascending=False)

---

### Charts

#### Troops, temperatures, cities and annotations

In [11]:
troops_chart = (
    alt.Chart(troops)
    .mark_trail()
    .encode(
        longitude="long:Q",
        latitude="lat:Q",
        size=alt.Size("survivors", scale=alt.Scale(range=[1, 50]), legend=None),
        detail="group:N",
        color=alt.Color(
            "direction:N",
            scale=alt.Scale(domain=["A", "R"], range=["#ebd2a8", "#000000"]),
            legend=None,
        ),
        tooltip=[
            alt.Tooltip("lat:Q", title="Latitude"),
            alt.Tooltip("long:Q", title="Longitude"),
            alt.Tooltip("survivors:Q", title="Survivors"),
        ],
    )
    .properties(width=900, height=300)
)

cities_text = (
    alt.Chart(cities)
    .mark_text(
        font="Didot",
        fontStyle="italic",
    )
    .encode(
        longitude="long:Q",
        latitude="lat:Q",
        text="city:N",
        size=alt.value(13),
    )
    .properties(width=900, height=300)
)

temp_chart = (
    alt.Chart(temps)
    .mark_line(size=4, strokeDash=[1, 1])
    .encode(
        x=alt.X(
            "long:Q",
            axis=alt.Axis(
                title="",
                grid=False,
                labels=False,
                ticks=False,
                domain=False,
            ),
            sort=None,
        ),
        y=alt.Y(
            "temp_c:Q",
            axis=alt.Axis(
                title="°C",
                ticks=False,
                tickCount=4,
                domain=False,
                orient="right",
                values=[-10, -20, -30, -40],
            ),
        ),
        color=alt.value("#c6c6c6"),
    )
    .properties(
        width=900,
        height=100,
        title="Temperatures during Napoleon’s retreat: Moscow to the Niemen River, near Kaunas, Lithuania",
    )
)

temp_annotations = pd.DataFrame(
    [
        {"long": 37.6, "temp_c": 0, "label": "Oct. 18: 0°C"},
        {"long": 36, "temp_c": 0, "label": "Oct. 24: 0°C"},
        {"long": 29.2, "temp_c": -13.75, "label": "Nov 24: -14°C"},
        {"long": 28.5, "temp_c": -25, "label": "Nov 28: -25°C"},
        {"long": 32.0, "temp_c": -26.25, "label": "Nov 14: -26°C"},
        {"long": 33.2, "temp_c": -11, "label": "Nov 9: -11°C"},
        {"long": 27.2, "temp_c": -30, "label": "Dec. 1: -30°C"},
        {"long": 26.7, "temp_c": -37.5, "label": "Dec. 6: -37.5°C"},
        {"long": 25.3, "temp_c": -32.5, "label": "Dec. 7: -32.5°C"},
    ]
)

annotations_chart = (
    alt.Chart(temp_annotations)
    .mark_text(
        align="center",
        baseline="bottom",
        font="Didot",
        fontStyle="italic",
        dx=0,
    )
    .encode(x="long:Q", y="temp_c:Q", text="label:N", size=alt.value(11))
)

combined_temp_chart = temp_chart + annotations_chart

final_chart = alt.vconcat(
    troops_chart + cities_text,
    combined_temp_chart,
    spacing=5,
).configure_view(strokeOpacity=0)

final_chart = (
    final_chart.configure_axis(labelFont="Didot", titleFont="Didot")
    .properties(
        title={
            "text": "Graphic representation of the French Army’s successive losses in men during the Russian campaign 1812-1813, by Charles Joseph Minard.",
            "subtitle": [
                "Color indicates direction: Beige for advance, black for retreat. Size represents troop strength.",
            ],
        }
    )
    .configure(font="Didot")
    .configure_title(
        font="Didot",
        fontSize=17,
        fontStyle="italic",
    )
)

---

### The result

In [12]:
final_chart

alt.VConcatChart(...)